# Sample Code to Run TS-LGB
This code shows how to use TS-LGB, a two-step algorithm for survival analysis with LightGBM.

## Load Data

In [11]:
# Prerequisite: pip install SurvSet
from SurvSet.data import SurvLoader

loader = SurvLoader()
df, _ = loader.load_dataset(ds_name="flchain").values()
print(df.head())

   pid  event  time  num_age  num_kappa  num_lambda  num_flc_grp  \
0    0      1    85       97       5.70       4.860           10   
1    1      1  1281       92       0.87       0.683            1   
2    2      1    69       94       4.36       3.850           10   
3    3      1   115       92       2.42       2.220            9   
4    4      1  1039       93       1.32       1.690            6   

   num_creatinine  num_sample_yr fac_sex  fac_chapter fac_sample_yr fac_mgus  
0             1.7           1997       F  Circulatory          1997        N  
1             0.9           2000       F    Neoplasms          2000        N  
2             1.4           1997       F  Circulatory          1997        N  
3             1.0           1996       F  Circulatory          1996        N  
4             1.1           1996       F  Circulatory          1996        N  


## Compute boundaries and label

In [12]:
import numpy as np

def compute_class(boundaries, num_risks, t, e):
    e = e.astype(int)
    idx = np.searchsorted(boundaries, t.reshape(-1, 1), side="right")
    return (idx.reshape(-1) - 1) * num_risks + e

lb = df["time"].min()
ub = df["time"].max()
boundaries = np.arange(lb, ub, 33)
print("boundaries", boundaries)
label = compute_class(boundaries, 2, df["time"].values, df["event"].values)
print("label", label)

boundaries [   0   33   66   99  132  165  198  231  264  297  330  363  396  429
  462  495  528  561  594  627  660  693  726  759  792  825  858  891
  924  957  990 1023 1056 1089 1122 1155 1188 1221 1254 1287 1320 1353
 1386 1419 1452 1485 1518 1551 1584 1617 1650 1683 1716 1749 1782 1815
 1848 1881 1914 1947 1980 2013 2046 2079 2112 2145 2178 2211 2244 2277
 2310 2343 2376 2409 2442 2475 2508 2541 2574 2607 2640 2673 2706 2739
 2772 2805 2838 2871 2904 2937 2970 3003 3036 3069 3102 3135 3168 3201
 3234 3267 3300 3333 3366 3399 3432 3465 3498 3531 3564 3597 3630 3663
 3696 3729 3762 3795 3828 3861 3894 3927 3960 3993 4026 4059 4092 4125
 4158 4191 4224 4257 4290 4323 4356 4389 4422 4455 4488 4521 4554 4587
 4620 4653 4686 4719 4752 4785 4818 4851 4884 4917 4950 4983 5016 5049
 5082 5115 5148 5181 5214]
label [  5  77   5 ... 150 300 242]


## One-hot Encoding

In [13]:
from sklearn.preprocessing import LabelEncoder

cols_cat = []
for col in df.columns:
    if col.startswith("fac_"):
        cols_cat.append(col)

le = LabelEncoder()
for col in cols_cat:
    le = le.fit(df[col])
    df[col]=le.transform(df[col])
print(df.head())

   pid  event  time  num_age  num_kappa  num_lambda  num_flc_grp  \
0    0      1    85       97       5.70       4.860           10   
1    1      1  1281       92       0.87       0.683            1   
2    2      1    69       94       4.36       3.850           10   
3    3      1   115       92       2.42       2.220            9   
4    4      1  1039       93       1.32       1.690            6   

   num_creatinine  num_sample_yr  fac_sex  fac_chapter  fac_sample_yr  \
0             1.7           1997        0            1              2   
1             0.9           2000        0           12              5   
2             1.4           1997        0            1              2   
3             1.0           1996        0            1              1   
4             1.1           1996        0            1              1   

   fac_mgus  
0         0  
1         0  
2         0  
3         0  
4         0  


## Split dataset

In [14]:
from sklearn.model_selection import train_test_split

# Drop the columns that are not needed for training
df.drop(columns=["pid", "time", "event"], inplace=True)

# Split the data into training and testing sets
df_train, df_test, label_train, label_test = train_test_split(df, label,random_state=42,test_size=0.2) 
print(df_train.shape, df_test.shape)
print(label_train.shape, label_test.shape)

(6299, 10) (1575, 10)
(6299,) (1575,)


## Train model with LightGBM

In [15]:
import lightgbm as lgb

lgb_model = lgb.LGBMClassifier(n_estimators=100, random_state=0)
lgb_model.fit(df_train, label_train, categorical_feature=cols_cat)
y_pred = lgb_model.predict_proba(df_test)
print("y_pred", y_pred, y_pred.shape)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000337 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 644
[LightGBM] [Info] Number of data points in the train set: 6299, number of used features: 10
[LightGBM] [Info] Start training from score -5.975557
[LightGBM] [Info] Start training from score -4.986946
[LightGBM] [Info] Start training from score -7.138708
[LightGBM] [Info] Start training from score -5.803707
[LightGBM] [Info] Start training from score -8.054999
[LightGBM] [Info] Start training from score -5.612652
[LightGBM] [Info] Start training from score -5.803707
[LightGBM] [Info] Start training from score -8.054999
[LightGBM] [Info] Start training from score -5.975557
[LightGBM] [Info] Start training from score -8.748146
[LightGBM] [Info] Start training from score -6.350251
[LightGBM] [Info] Start training from score -7.138708
[